### Setup and Dependencies
This section handles the initial setup, including installing necessary libraries and configuring external services.

In [1]:
!pip install streamlit pyngrok joblib scikit-learn xgboost lightgbm catboost --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 118.3 MB/s eta 0:00:00


This cell installs all the required Python libraries for the project, including `streamlit` for building the web application, `pyngrok` for exposing the local server to the internet, `joblib` for model persistence, and `scikit-learn`, `xgboost`, `lightgbm`, and `catboost` (although only `joblib` is directly used in the app, these are common ML dependencies).

`--quiet` is used to suppress verbose output during installation.

### Streamlit Application Definition
This cell defines the Python code for the Streamlit web application. It includes functionalities such as:

*   **User Configuration**: `MODEL_PATH` (where the trained machine learning model is stored) and `BACKGROUND_PATH_OR_URL` (for the app's visual background).
*   **Background Setter**: A function to apply a background image from a local path or a URL.
*   **Model Loader**: A function to load the pre-trained `joblib` model.
*   **Streamlit UI**: Sets up the page configuration, displays a header, creates input fields for prediction (e.g., N, P, K, temperature, crop type), a prediction button, and a section to display prediction history.

The `app_code` string containing the entire Streamlit script is then written to a file named `app.py` in the current directory.

In [13]:
app_code = r'''
# app.py — Final version (white bar for header, single prediction UI)
import streamlit as st
import pandas as pd
import numpy as np
import joblib, os, base64

# ---------- USER CONFIG ----------
MODEL_PATH = "/content/drive/MyDrive/Crop_Prediction_Project/best_model_LinearRegression.joblib"
BACKGROUND_PATH_OR_URL = "https://wallpaperaccess.com/full/736046.jpg"
# ---------------------------------

st.set_page_config(page_title="Yield Predictor", layout="wide", page_icon="🌾")

# ---------- BACKGROUND IMAGE ----------
def set_background(image_path_or_url, overlay_rgba="rgba(255,255,255,0.15)"):
    """Sets page background with optional overlay"""
    try:
        if image_path_or_url.startswith("http"):
            css = f"""
            <style>
            .stApp {{
              background: linear-gradient({overlay_rgba},{overlay_rgba}), url("{image_path_or_url}");
              background-size: cover;
              background-position: center;
              background-attachment: fixed;
            }}
            .content-box {{
              background: rgba(255,255,255,0.85);
              padding: 20px;
              border-radius: 10px;
              box-shadow: 0 2px 8px rgba(0,0,0,0.15);
            }}
            </style>
            """
            st.markdown(css, unsafe_allow_html=True)
            return True
        elif os.path.exists(image_path_or_url):
            with open(image_path_or_url, "rb") as f:
                data = f.read()
            b64 = base64.b64encode(data).decode()
            css = f"""
            <style>
            .stApp {{
              background: linear-gradient({overlay_rgba},{overlay_rgba}), url("data:image/jpg;base64,{b64}");
              background-size: cover;
              background-position: center;
              background-attachment: fixed;
            }}
            .content-box {{
              background: rgba(255,255,255,0.85);
              padding: 20px;
              border-radius: 10px;
              box-shadow: 0 2px 8px rgba(0,0,0,0.15);
            }}
            </style>
            """
            st.markdown(css, unsafe_allow_html=True)
            return True
    except Exception as e:
        st.warning(f"Background image failed: {e}")
    return False

set_background(BACKGROUND_PATH_OR_URL)

# ---------- LOAD MODEL ----------
def load_model(path):
    if not os.path.exists(path):
        return None, f"Model not found at {path}"
    try:
        return joblib.load(path), None
    except Exception as e:
        return None, str(e)

model, err = load_model(MODEL_PATH)
if model is None:
    st.error(f"❌ Model could not be loaded: {err}")
    st.stop()

# ---------- HEADER ----------
st.markdown("""
    <div style="background-color:white; border-radius:10px; padding:15px; box-shadow:0 2px 8px rgba(0,0,0,0.15); text-align:center; margin-bottom:10px;">
        <h1 style="color:#0b5f2b; font-weight:700; margin:0;">🌾 Yield Predictor</h1>
        <p style="color:#2e2e2e; font-size:16px; margin-top:4px;">
            Enter your crop and soil parameters to estimate yield instantly.
        </p>
    </div>
""", unsafe_allow_html=True)

# ---------- INPUT FORM ----------
st.markdown("<div class='content-box'>", unsafe_allow_html=True)
st.subheader("Input Parameters")

# Editable feature list — adjust if your model expects different ones
numeric_features = ["N", "P", "K", "temperature", "humidity", "ph", "rainfall"]
categorical_features = ["Crop", "Season", "Soil_Type"]

inputs = {}
col1, col2 = st.columns(2)
with col1:
    for f in numeric_features:
        inputs[f] = st.number_input(f, value=0.0, step=0.1, format="%.2f")
with col2:
    for f in categorical_features:
        inputs[f] = st.text_input(f, "")

if st.button("🔮 Predict Yield"):
    sample = pd.DataFrame([inputs])
    try:
        pred = model.predict(sample)[0]
        st.markdown(
            f"<div style='text-align:center; margin-top:20px; padding:15px; "
            f"border-radius:10px; background:#eaffea; color:#0b5f2b; font-size:22px;'>"
            f"<b>Predicted Yield:</b> {pred:.3f} t/ha</div>",
            unsafe_allow_html=True,
        )
        # Store history
        if "history" not in st.session_state:
            st.session_state["history"] = []
        rec = sample.copy()
        rec["Predicted_Yield"] = pred
        st.session_state["history"].append(rec)
    except Exception as e:
        st.error(f"⚠️ Prediction failed: {e}")

st.markdown("</div>", unsafe_allow_html=True)

# ---------- HISTORY SECTION ----------
st.markdown("<div class='content-box'>", unsafe_allow_html=True)
st.subheader("Prediction History")

if "history" in st.session_state and len(st.session_state["history"]) > 0:
    hist_df = pd.concat(st.session_state["history"], ignore_index=True)
    st.dataframe(hist_df.tail(5))
    csv = hist_df.to_csv(index=False).encode("utf-8")
    st.download_button("⬇️ Download Prediction History", data=csv, file_name="prediction_history.csv")
else:
    st.info("No predictions yet.")
st.markdown("</div>", unsafe_allow_html=True)

# ---------- FOOTER ----------
st.markdown("""
    <hr style="opacity:0.3;">
    <p style="text-align:center; color:white; font-size:13px;">
    Designed by <b>Manohar Vinay Mududundi</b> | Crop Yield Prediction UI 2025
    </p>
""", unsafe_allow_html=True)

'''
with open("app.py", "w", encoding="utf-8") as f:
    f.write(app_code)
print("✅ Streamlit app created successfully!")


✅ Streamlit app created successfully!


### Ngrok Authentication
This cell configures `pyngrok` by setting the `NGROK_AUTHTOKEN`. This token is essential for authenticating with the ngrok service, which allows your locally running Streamlit application to be exposed to the internet via a public URL. The token is stored as a variable and a warning is issued if it's not replaced with a valid one.

In [19]:
from pyngrok import ngrok
import os
from google.colab import userdata

NGROK_AUTHTOKEN = userdata.get('NGROK_AUTHTOKEN')


if NGROK_AUTHTOKEN is None or NGROK_AUTHTOKEN == "YOUR_NGROK_AUTHTOKEN":
    print("⚠️ NGROK_AUTHTOKEN not found in Colab Secrets. Please add your ngrok authtoken to Colab Secrets (named 'NGROK_AUTHTOKEN').")
    print("Execution halted. Please add your token and rerun this cell.")

else:
    ngrok.set_auth_token(NGROK_AUTHTOKEN)
    print("✅ ngrok authtoken set from Colab Secrets.")

✅ ngrok authtoken set from Colab Secrets.


### Launch Streamlit App with Ngrok
This cell performs two main actions:

1.  **Launches Streamlit**: It starts the Streamlit application (`app.py`) as a subprocess, which typically runs on port `8501`. A command `fuser -k 8501/tcp` is used to kill any process currently using port 8501 to ensure the Streamlit app can start cleanly.
2.  **Creates Public URL**: It then uses `pyngrok` to establish a tunnel to the Streamlit app running on port `8501`, generating a public URL. This URL allows external users to access the Streamlit application running in this Colab environment.

In [14]:
from pyngrok import ngrok
import time, subprocess

# Kill any existing streamlit instances
!fuser -k 8501/tcp || echo "Port 8501 free"

print("🚀 Launching Streamlit...")
proc = subprocess.Popen(["streamlit", "run", "app.py"], stdout=subprocess.PIPE)
time.sleep(5)

# Create public URL
public_url = ngrok.connect(8501)
print("🌍 Your Streamlit App is Live Here:")
print(public_url)


8501/tcp:             5510
🚀 Launching Streamlit...


🌍 Your Streamlit App is Live Here:
NgrokTunnel: "https://opal-mirky-noncontroversially.ngrok-free.dev" -> "http://localhost:8501"
